In [2]:
import numpy as np
import zarr

def load_and_reshape(filename, nx_m, ny_m, nz_m):
    np.set_printoptions(threshold=np.inf, edgeitems=10)
    data = zarr.open(filename, 'r')[:]
    data = data.astype(np.uint8)
    #print(data)
    # print(data.size)
    #20, 9 ,11
    print(data.shape)
    #originally in z, y, x
    data= np.transpose(data, (1, 0, 2))
    #2, 1, 0
    # y x z
    # x z y
    print(data.shape)
    modelreshape = np.reshape(data, (ny_m, nz_m, nx_m))
    print(modelreshape.shape)
    # print(data.shape)
    #y, z, x
    return modelreshape

# def load_and_reshape_eye(filename, nx_m, ny_m, nz_m):
#     np.set_printoptions(threshold=np.inf, edgeitems=10)
#     data = zarr.open(filename, 'r')[:]
#     #print(data)
#     #data= np.transpose(data, (1, 0, 2))
#     # y x z
#     # x z y
#     # print(data.shape)
#     modelreshape = np.reshape(data, (ny_m, nz_m, nx_m))
#     # print(data.shape)
#     return modelreshape

# Target center coordinates
target_center = [320 // 2, 115 // 2, 300 // 2]
mcx = 320
mcy = 115
mcz = 300

# Initialize the merged model with appropriate size (adjust this size as needed)
merged_size = (320, 115, 300)  # Example size, ensure it's large enough to hold all data
model3D_new = np.zeros(merged_size, dtype=np.uint8)
model3D_new = np.reshape(model3D_new, (mcy, mcz, mcx))

def merge_zarr_to_center(model, data, center, target_center):
    offset_x = target_center[0] - center[0]
    offset_y = target_center[1] - center[1]
    offset_z = target_center[2] - center[2]



    #x, z, y 
    ny_m, nz_m, nx_m = data.shape
    #ny = x
    #nz = z 
    #nx = y
    #333, 774, 415
    print("model shape")
    print(model.shape)
    print("data shape")
    print(data.shape)
    #12, 37, 55 
    print("coordinates")
    #x, y, z
    print(nx_m, ny_m, nz_m)
    #93 114 44
    #93 + 55 = 148
    #114 + 12 = 126
    #44 + 37 = 81
    #55 12 37

    for i in range(nx_m):
        for j in range(ny_m):
            for k in range(nz_m):
                if data[j, k, i] != 0:
                    # print("offset")
                    # print(offset_y + j, offset_z + k, offset_x + i)
                    model[offset_y + j, offset_z + k, offset_x + i] = data[j, k, i]
    #23 - 13 
    return model

# for i in range(nx_m):
#     for j in range(ny_m):
#         for k in range(nz_m):
#             if PB[j, k, i] != 0:
#                 model3D_new[cy - (ny_m // 2) + j, cz - (nz_m // 2) + k, cx - (nx_m // 2) + i] = PB[j, k, i]

#center to be merged
# cx = 55 // 2
# cy = 12 // 2
# cz = 37 // 2

nx_m = 55
ny_m = 12
nz_m = 37
PG = load_and_reshape('Cranial Nerve II - Optic_55x12x37.zarr', nx_m, ny_m, nz_m)
new_center = [nx_m // 2, ny_m // 2, nz_m // 2]
# print("The dtype of PG is:", PG.dtype)
model3D_new = merge_zarr_to_center(model3D_new, PG, new_center, target_center)
# model3D_new = np.copy(PG)
old_center = [nx_m // 2, ny_m // 2, nz_m // 2]

# print("Shape of model3D_new:", model3D_new.shape)
#good

# target_center[0] = target_center[0] - (nx_m//2) - 55
# target_center[1] = target_center[1] + (ny_m//2)
# target_center[2] = target_center[2] + (nz_m//2) + (11//2)
#offset for x was 27 ->54
#offset for y was 6 -> 12
#offset for z was 18 -> 36
#now y z x
#x, z, y 
#20, 11, 9
#j, k i,
#11, 9, 20?????
nx_m = 20
ny_m = 9
nz_m = 11
OC = load_and_reshape('Optic Chiasm_20x9x11.zarr', nx_m, ny_m, nz_m)
new_center = [nx_m // 2, ny_m // 2, nz_m // 2]
target_center[1] = target_center[1] + old_center[1]
target_center[2] = target_center[2] + old_center[2] + new_center[2]
model3D_new = merge_zarr_to_center(model3D_new, OC, new_center, target_center)
second_old_center = [nx_m // 2, ny_m // 2, nz_m // 2]

nx_m = 27
ny_m = 7
nz_m = 14
OT = load_and_reshape('Optic Tract_27x7x14.zarr', nx_m, ny_m, nz_m)
new_center = [nx_m // 2, ny_m // 2, nz_m // 2]
target_center[1] = target_center[1] + new_center[1]
target_center[2] = target_center[2] + new_center[2]
model3D_new = merge_zarr_to_center(model3D_new, OT, new_center, target_center)
third_old_center = [nx_m // 2, ny_m // 2, nz_m // 2]
#target center = 75
#old center = 27, 6, 18

nx_m = 27
ny_m = 27
nz_m = 24
HRsclera = load_and_reshape('Eye_Healthy_Right_Sclera_27x27x24.zarr', nx_m, ny_m, nz_m)

# HRsclera = np.flip(HRsclera, axis=(0, 1))

new_center = [nx_m // 2, ny_m // 2, nz_m // 2]
target_center[0] = target_center[0] - old_center[0]
target_center[1] = target_center[1] - old_center[1] - third_old_center[1]
target_center[2] = target_center[2] - 2*old_center[2] - second_old_center[2] - new_center[2]

model3D_new = merge_zarr_to_center(model3D_new, HRsclera,  new_center, target_center)
right_old_center = [nx_m // 2, ny_m // 2, nz_m // 2]

nx_m = 26
ny_m = 26
nz_m = 23
HRchoroid = load_and_reshape('Eye_Healthy_Right_Choroid_26x26x23.zarr', nx_m, ny_m, nz_m)

# HRsclera = np.flip(HRsclera, axis=(0, 1))

new_center = [nx_m // 2, ny_m // 2, nz_m // 2]
# target_center[0] = target_center[0] - old_center[0]
# target_center[1] = target_center[1] - old_center[1] - third_old_center[1]
# target_center[2] = target_center[2] - 2*old_center[2] - second_old_center[2] - new_center[2]

model3D_new = merge_zarr_to_center(model3D_new, HRchoroid, new_center, target_center)
# right_old_center = [nx_m // 2, ny_m // 2, nz_m // 2]


nx_m = 25
ny_m = 25
nz_m = 23
DLsclera = load_and_reshape('Eye_Left_Degenerated_Sclera_25x25x23.zarr', nx_m, ny_m, nz_m)

# HRsclera = np.flip(HRsclera, axis=(0, 1))

new_center = [nx_m // 2, ny_m // 2, nz_m // 2]
target_center[0] = target_center[0] + 2*old_center[0]
target_center[2] = target_center[2] + right_old_center[2] - new_center[2]
model3D_new = merge_zarr_to_center(model3D_new, DLsclera, new_center, target_center)



# for i in range(nx_m):
#     for j in range(ny_m):
#         for k in range(nz_m):
#             if PB[j, k, i] != 0:
#                 model3D_new[cy - (ny_m // 2) + j, cz - (nz_m // 2) + k, cx - (nx_m // 2) + i] = PB[j, k, i]


# for i in range(nx_m):
#     for j in range(ny_m):
#         for k in range(nz_m):
#             if OC[j, k, i] != 0:
#                 model3D_new[cy - (ny_m // 2) + j, cz - (nz_m // 2) + k, cx - (nx_m // 2) + i] = OC[j, k, i]
# for j in range(model3D_new.shape[2]):
#     print(f"(:,:,{j+1}) =\n\n",  model3D_new[:, :, j])

# zarr.save('Merged_Model.zarr', model3D_new)

# model3D_new = zarr.load('Merged_Model.zarr')

# for j in range(model3D_new.shape[2]):
#     print(f"(:,:,{j+1}) =\n\n",  model3D_new[:, :, j])

# with open(f"MergedModel.txt", 'w') as f:
#     f.write("Shape of the loaded Zarr array:\n")
#     f.write(str(model3D_new.shape) + '\n')
#     f.write("Contents of the Zarr array:\n")
#     np.savetxt(f, model3D_new.reshape(-1, model3D_new.shape[-1]), fmt='%d', delimiter='\t')

# print(f"Contents of the Zarr array saved to MergedModel.txt")

#save to .model file
#already in y, z, x
#13 12 16
#wrong for now
# model_shape = model3D_new.shape
# output_model = model3D_new.reshape(model_shape[1] * model_shape[2], model_shape[0])
# np.savetxt('Head_NEW_480_480_350.model', output_model, delimiter=' ', fmt='%d')

# Save to .raw file
#z, x, y 2 0 1
# if isinstance(model3D_new, np.ndarray):
#     print("model3D_new is a NumPy array.")
# else:
#     print("model3D_new is not a NumPy array.")
# print(model3D_new.shape)
# print("The dtype of model3D_new is:", model3D_new.dtype)
# model3D_new = model3D_new.astype(np.int64)
# print("The dtype of model3D_new is:", model3D_new.dtype)
#201 for function
#102 else
#x, z, y
#z, y, x
#y, z, x
#1 0 2
#z, y, x
modelraw = np.transpose(model3D_new, (1, 0, 2))
print(modelraw.shape)
# for j in range(modelraw.shape[2]):
#     print(f"(:,:,{j+1}) =\n\n",  modelraw[:, :, j])
# print(modelraw)

with open('UINT8FinalEye_480_480_3503.raw', 'wb') as fid:
    modelraw.astype('uint8').tofile(fid)
#astype('uint8')


(37, 12, 55)
(12, 37, 55)
(12, 37, 55)
model shape
(115, 300, 320)
data shape
(12, 37, 55)
coordinates
55 12 37
(11, 9, 20)
(9, 11, 20)
(9, 11, 20)
model shape
(115, 300, 320)
data shape
(9, 11, 20)
coordinates
20 9 11
(14, 7, 27)
(7, 14, 27)
(7, 14, 27)
model shape
(115, 300, 320)
data shape
(7, 14, 27)
coordinates
27 7 14
(24, 27, 27)
(27, 24, 27)
(27, 24, 27)
model shape
(115, 300, 320)
data shape
(27, 24, 27)
coordinates
27 27 24
(23, 26, 26)
(26, 23, 26)
(26, 23, 26)
model shape
(115, 300, 320)
data shape
(26, 23, 26)
coordinates
26 26 23
(23, 25, 25)
(25, 23, 25)
(25, 23, 25)
model shape
(115, 300, 320)
data shape
(25, 23, 25)
coordinates
25 25 23
(300, 115, 320)
